### Now that we have a datawarehouse with data ready to analyize, we calculate the payouts of using 538 to bet on certain games

In [2]:
## imports
import pandas as pd
import sqlite3
from sqlite3 import Error


In [3]:
## Lets build functions that gives percentages based off of a moneyline
def get_pct_from_moneyline(moneyline):
    
    moneyline = int(moneyline)
    
    ## If the moneyline is negative
    if moneyline < 0:
        
        pct = (-1 * moneyline) / ((-1 * moneyline) + 100)

    ## If the moneyline is positive
    elif moneyline > 0:
        
        pct = 100 / (moneyline + 100)

    pct = 100 * round(pct, 4)
    return pct

In [4]:
## We get our data from the data warehouse
data_warehouse = r"C:\Users\arbis\Projects\nate_silver_gets_me_money\database\dk_538_games_dw.db" 
con = None
try:
    con = sqlite3. connect(data_warehouse)
except Error as e:
    print(e)

## we get future games
game_dw = pd.read_sql_query("SELECT * FROM game_dw WHERE date(game_date) >= date('now');", con)

con.close()

game_dw.head()

,game_date,away_team,home_team,away_pct_538,home_pct_538,away_moneyline_dk,home_moneyline_dk
0,2021-09-09,Dodgers,Cardinals,0.62,0.38,-170,150
1,2021-09-09,White Sox,Athletics,0.45,0.55,130,-150
2,2021-09-09,Twins,Indians,0.43,0.57,125,-145
3,2021-09-09,Mets,Marlins,0.58,0.42,-180,155
4,2021-09-09,Rockies,Phillies,0.43,0.57,160,-190


In [5]:
## then apply our odds function to our draftkings moneyline to get our draftkings raw probability percentage
game_dw["raw_home_odds_dk"] = game_dw["home_moneyline_dk"].apply(get_pct_from_moneyline)
game_dw["raw_away_odds_dk"] = game_dw["away_moneyline_dk"].apply(get_pct_from_moneyline)

## sports book odds add up to 100 + the padding percentage, or their estimated profit margin
game_dw["dk_total_padding_val"] = game_dw["raw_home_odds_dk"] + game_dw["raw_away_odds_dk"] - 100

## This padding percentage is (THEORETICALLY) added equally on both sides, or at least in very close amounts
## We rescale the raw percentages back to a 100% total scale to get our final percentage
game_dw["home_odds_dk"] = game_dw["raw_home_odds_dk"] / (game_dw["raw_home_odds_dk"] + game_dw["raw_away_odds_dk"])
game_dw["home_odds_dk"] = round(game_dw["home_odds_dk"], 4)

game_dw["away_odds_dk"] = game_dw["raw_away_odds_dk"] / (game_dw["raw_home_odds_dk"] + game_dw["raw_away_odds_dk"])
game_dw["away_odds_dk"] = round(game_dw["away_odds_dk"], 4)

## Now lets look at differences between 538's probabilities and draftkings
game_dw['diff_pct_home'] = game_dw["home_pct_538"] - game_dw["home_odds_dk"]
game_dw['diff_pct_away'] = game_dw["away_pct_538"] - game_dw["away_odds_dk"]

game_dw.head(5)

,game_date,away_team,home_team,away_pct_538,home_pct_538,away_moneyline_dk,home_moneyline_dk,raw_home_odds_dk,raw_away_odds_dk,dk_total_padding_val,home_odds_dk,away_odds_dk,diff_pct_home,diff_pct_away
0,2021-09-09,Dodgers,Cardinals,0.62,0.38,-170,150,40.00,62.96,2.96,0.3885,0.6115,-0.0085,0.0085
1,2021-09-09,White Sox,Athletics,0.45,0.55,130,-150,60.00,43.48,3.48,0.5798,0.4202,-0.0298,0.0298
2,2021-09-09,Twins,Indians,0.43,0.57,125,-145,59.18,44.44,3.62,0.5711,0.4289,-0.0011,0.0011
3,2021-09-09,Mets,Marlins,0.58,0.42,-180,155,39.22,64.29,3.51,0.3789,0.6211,0.0411,-0.0411
4,2021-09-09,Rockies,Phillies,0.43,0.57,160,-190,65.52,38.46,3.98,0.6301,0.3699,-0.0601,0.0601


In [6]:
def bet_outcome(bet, moneyline):
    '''This function calculates the outcome of making bets, taking in a bet amount and a moneyline'''
    moneyline = int(moneyline)
    
    ## if the moneyline is negative (betting favortie)
    if moneyline < 0:
        
        outcome = (bet  100) / abs(moneyline)
        
    ## if the moneyline is positive (underdog)
    if moneyline > 0:
        
        outcome = (bet * moneyline) / 100

    outcome = bet + round(outcome , 2)
        
    return outcome

SyntaxError: invalid syntax (<ipython-input-6-4e993b7fd8a9>, line 9)

In [ ]:
## We get our winnings columns by applying our bet_ouctome function to the original moneyline
game_dw["winnings_on_1_dollar_home"] = game_dw["home_moneyline_dk"].apply(lambda ml: bet_outcome(1, ml))
game_dw["winnings_on_1_dollar_away"] = game_dw["away_moneyline_dk"].apply(lambda ml: bet_outcome(1, ml))

## And get our expected outcome by multipling the winnings by the probability
game_dw["expected_outcome_home_538"] = (game_dw["winnings_on_1_dollar_home"] * game_dw["home_pct_538"]) - 1
game_dw["expected_outcome_away_538"] = (game_dw["winnings_on_1_dollar_away"] * game_dw["away_pct_538"]) - 1


game_dw[["game_date","home_team","away_team","expected_outcome_home_538","expected_outcome_away_538", "away_moneyline_dk"]].head()

Couple of things to notice:
 - For some games, both sides have a positive expected value
 - If you add the draft king percentabes together, you will get a value higher than 100
 - This is because draft kings pads their percentages, the generally add 1-2% to each outcome, this becomes their profit margin

To get account for this percentage padding, we adjust our expected outcome by subtracting half of the total percentage padding to each 538 prediction, and recalculate our expected value

In [ ]:
game_dw["adj_expected_outcome_away_538"] = game_dw["winnings_on_1_dollar_away"] * (game_dw["away_pct_538"] - game_dw['dk_total_padding_val'] / 100 / 2) - 1
game_dw["adj_expected_outcome_home_538"] = game_dw["winnings_on_1_dollar_home"] * (game_dw["home_pct_538"] - game_dw['dk_total_padding_val'] / 100 / 2) - 1

game_dw.head()

#### Here is an example of games that have a potentially positive expected outcome

In [ ]:
game_dw.loc[(game_dw['adj_expected_outcome_away_538'] > 0) | (game_dw['adj_expected_outcome_home_538'] > 0)][["game_date", "away_team", "home_team", "away_pct_538", "home_pct_538", "away_moneyline_dk", "home_moneyline_dk", "adj_expected_outcome_away_538", "adj_expected_outcome_home_538"]]

In [ ]:
# game_dw.to_csv('sample_calculated_data.csv', index = False)